# Experiment Analysis: Controller parameter stability search
Perform shocks of ETH price to test controller parameter stability, without stochastic processes.

* See `experiments/system_model_v3/experiment_shocks.py`

In [ ]:
%load_ext autotime
# danlessa was here

# Setup and Dependencies

In [2]:
# Set project root folder, to enable importing project files from subdirectories
from pathlib import Path
import os

path = Path().resolve()
root_path = str(path).split('notebooks')[0]
os.chdir(root_path)

# Force reload of project modules, sometimes necessary for Jupyter kernel
%load_ext autoreload
%autoreload 2

# Display framework versions for easy debugging
%pip show cadCAD
%pip show radcad

Name: cadCAD
Version: 0.4.23
Summary: cadCAD: a differential games based simulation software package for research, validation, and         Computer Aided Design of economic systems
Home-page: https://github.com/cadCAD-org/cadCAD
Author: Joshua E. Jodesty
Author-email: joshua@block.science
License: LICENSE.txt
Location: /home/bscholtz/workspace/reflexer/venv/lib/python3.8/site-packages
Requires: fn, funcy, pandas, pathos
Required-by: cadCAD-machine-search
Note: you may need to restart the kernel to use updated packages.
Name: radcad
Version: 0.5.6
Summary: A cadCAD implementation, for dynamical systems modelling & simulation
Home-page: None
Author: Benjamin Scholtz
Author-email: ben@bitsofether.com
License: None
Location: /home/bscholtz/workspace/reflexer/venv/lib/python3.8/site-packages
Requires: boto3, ray, pandas, pathos, tables
Required-by: 
Note: you may need to restart the kernel to use updated packages.
time: 1.65 s (started: 2021-02-12 10:50:41 +02:00)


In [3]:
# danlessa was here
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
time: 425 ms (started: 2021-02-12 10:50:43 +02:00)


In [4]:
# Import all shared dependencies and setup
#from shared import *

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import plotly.io as pio
#pio.renderers.default = "png"
from pprint import pprint

time: 538 ms (started: 2021-02-12 10:50:43 +02:00)


In [5]:
# Update dataframe display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)
pd.options.plotting.backend = "plotly"

time: 25.1 ms (started: 2021-02-12 10:50:44 +02:00)


# Load Dataset

In [6]:
processed_results = 'experiments/system_model_v3/experiment_monte_carlo/processed_results.hdf5'

time: 10.3 ms (started: 2021-02-12 10:50:44 +02:00)


In [7]:
df = pd.read_hdf(processed_results, key='results')
df

,timedelta,cumulative_time,timestamp,eth_price,liquidity_demand,liquidity_demand_mean,eth_collateral,eth_locked,eth_freed,principal_debt,rai_drawn,rai_wiped,system_revenue,stability_fee,market_price,market_price_twap,target_price,target_rate,eth_return,eth_gross_return,expected_market_price,expected_debt_price,error_star,error_star_integral,market_slippage,RAI_balance,ETH_balance,UNI_supply,simulation,subset,run,substep,timestep,eth_collateral_value,collateralization_ratio,ki,kp,liquidation_ratio,controller_enabled,control_period,liquidity_demand_shock,arbitrageur_considers_liquidation_ratio,rescale_target_price,target_price_scaled
0,0,0,2017-01-01 00:00:00,294.069151,1.000000,1.000000,154827.528922,154827.528922,0.000000,1.000000e+07,1.000000e+07,0.000000e+00,0.0,1.585490e-10,3.140000,0.000000,3.140000,0.0,0.000000,0.0,3.140000,3.14,0.000000,0.000000,0.000000,1.000000e+07,106777.606153,1.000000e+07,0,0,1,0,0,4.553000e+07,1.450000,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,4.553
1,3600,3600,2017-01-01 01:00:00,291.208252,906.674484,453.837242,154827.528922,154827.528922,0.000000,1.450000e+07,1.450000e+07,0.000000e+00,0.0,1.585490e-10,3.139432,0.000000,2.165517,0.0,-0.009729,1.0,3.231162,3.14,3.140000,5652.000000,0.000181,1.450091e+07,73703.760510,1.000000e+07,0,0,1,18,1,4.508705e+07,1.435893,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,3.140
2,3600,7200,2017-01-01 02:00:00,292.287792,-326.024615,63.906314,107503.536596,154827.528922,47323.992326,9.970037e+06,1.450000e+07,4.529963e+06,0.0,1.585490e-10,1.480124,0.000000,2.165517,0.0,0.003707,1.0,1.523521,3.14,3.140000,11458.000000,NaN,9.970618e+06,107288.025922,9.999775e+06,0,0,1,18,2,3.142197e+07,1.455375,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,3.140
3,3600,10800,2017-01-01 03:00:00,289.343698,209.658450,136.782382,107503.536596,154827.528922,47323.992326,1.000700e+07,1.453696e+07,4.529963e+06,0.0,1.585490e-10,3.145007,0.000000,2.165517,0.0,-0.010073,1.0,3.236929,3.14,3.140000,11616.000000,-1.124826,1.000779e+07,106890.741521,9.999775e+06,0,0,1,18,3,3.110547e+07,1.435395,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,3.140
4,3600,14400,2017-01-01 04:00:00,286.827771,-79.844839,28.468771,107503.536596,154827.528922,47323.992326,9.906201e+06,1.453696e+07,4.630759e+06,0.0,1.585490e-10,3.090459,0.000000,2.165517,0.0,-0.008695,1.0,3.181390,3.14,3.140000,11620.000000,0.017344,9.906911e+06,107982.421498,9.999775e+06,0,0,1,18,4,3.083500e+07,1.437392,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,3.140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8711142,3600,15537600,2017-06-29 20:00:00,333.003922,144.042027,11414.400180,43950.155901,164697.920768,120747.764867,3.228395e+06,6.056715e+07,5.733875e+07,0.0,1.585490e-10,4.599548,4.630135,3.140000,0.0,-0.004307,1.0,4.715210,3.14,-1.490135,-5514.485458,NaN,1.899010e+07,261063.604866,2.118011e+07,0,431,5,18,4316,1.463557e+07,1.443755,-2.000000e-10,5.000000e-06,1.45,False,25200.0,False,False,False,4.553
8711143,3600,15541200,2017-06-29 21:00:00,331.275043,-9229.101700,1092.649240,43950.155901,164697.920768,120747.764867,3.214490e+06,6.056715e+07,5.735266e+07,0.0,1.585490e-10,4.582382,4.640406,3.140000,0.0,-0.005192,1.0,4.698362,3.14,-1.500406,-5532.974094,0.003732,1.896697e+07,261382.975946,2.118011e+07,0,431,5,18,4317,1.455959e+07,1.442472,-2.000000e-10,5.000000e-06,1.45,False,25200.0,False,False,False,4.553
8711144,3600,15544800,2017-06-29 22:00:00,331.782012,110.399771,601.524506,43950.155901,164697.920768,120747.764867,3.197801e+06,6.056715e+07,5.736935e+07,0.0,1.585490e-10,4.565235,4.640406,3.140000,0.0,0.001530,1.0,4.683459,3.14,-1.500406,-5551.462730,0.003742,1.895039e+07,261612.340372,2.118011e+07,0,431,5,18,4318,1.458187e+07,1.452219,-2.000000e-10,5.000000e-06,1.45,False,25200.0,False,False,False,4.553
8711145,3600,15548400,2017-0

time: 5.86 s (started: 2021-02-12 10:50:44 +02:00)


# Process KPIs

In [8]:
df_kpis = df.copy()

time: 567 ms (started: 2021-02-12 10:50:50 +02:00)


In [9]:
# danlessa was here 2.5s -> 0.9s
cols = ['target_price', 'liquidation_ratio', 'rescale_target_price']
f = lambda x: (x['target_price'] * x['liquidation_ratio']) if x['rescale_target_price'] else x['target_price']
df_kpis['target_price_scaled'] = df_kpis[cols].parallel_apply(f, axis=1)
df_kpis['target_price_scaled'].head(10)

0    4.553
1    3.140
2    3.140
3    3.140
4    3.140
5    3.140
6    3.140
7    3.140
8    3.140
9    3.140
Name: target_price_scaled, dtype: float64

time: 11.3 s (started: 2021-02-12 10:50:50 +02:00)


## Stability

**Stability** threshold of system: defined as the maximum value for relative frequency of simulation runs that are unstable. Unstable is measured as fraction of runs where:
  - market price runs to infinity/zero (e.g. upper bound 10xPI; lower bound 0.10xPI if initial price is PI);
  - redemption price runs to infinity/zero (e.g. upper bound 10xPI; lower bound 0.10xPI if initial price is PI);
  - Uniswap liquidity (RAI reserve) runs to zero;
  - CDP position (total ETH collateral) runs to infinity/zero.

In [10]:
initial_target_price = df_kpis['target_price'].iloc[0]
initial_target_price

3.14

time: 13.1 ms (started: 2021-02-12 10:51:02 +02:00)


In [11]:
df_kpis[['market_price', 'target_price_scaled', 'RAI_balance', 'eth_collateral']].describe([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.90])

,market_price,target_price_scaled,RAI_balance,eth_collateral
count,8.711147e+06,8.711147e+06,8.711147e+06,8.711147e+06
mean,1.486891e+02,2.353824e+00,1.205188e+07,9.662898e+04
std,1.104772e+04,1.127780e+00,8.701426e+06,4.710633e+04
min,3.985592e-03,1.738892e-02,2.276185e+03,5.871264e+03
10%,4.823438e-01,3.890588e-01,5.131811e+06,3.396754e+04
20%,1.262839e+00,1.029916e+00,6.821149e+06,5.015548e+04
30%,2.053286e+00,1.837509e+00,8.071490e+06,7.021243e+04
40%,2.902159e+00,2.792804e+00,8.949617e+06,9.045166e+04
50%,3.350532e+00,3.140000e+00,9.900816e+06,1.044786e+05
60%,4.278496e+00,3.140000e+00,1.104458e+07,1.077841e+05


time: 1.08 s (started: 2021-02-12 10:51:02 +02:00)


In [12]:
df_stability = df_kpis.groupby(['subset', 'run'])

df_stability = df_stability.agg({
    'market_price': ['min', 'max'],
    'target_price_scaled': ['min', 'max'],
    'RAI_balance': ['min', 'max'],
    'eth_collateral': ['min', 'max'],
})
df_stability.columns = [
    'market_price_min', 'market_price_max',
    'target_price_min', 'target_price_max',
    'RAI_balance_min', 'RAI_balance_max',
    'eth_collateral_min', 'eth_collateral_max'
]
df_stability = df_stability.reset_index()

df_stability['stability_market_price'] = df_stability \
    .apply(lambda x: x['market_price_min'] >= 0.1*initial_target_price and x['market_price_max'] <= 10*initial_target_price, axis=1)

df_stability['stability_target_price'] = df_stability \
    .apply(lambda x: x['target_price_min'] >= 0.1*initial_target_price and x['target_price_max'] <= 10*initial_target_price, axis=1)

# TODO: discuss threshold
df_stability['stability_uniswap_liquidity'] = df_stability \
    .apply(lambda x: x['RAI_balance_min'] >= 500e3, axis=1)

# TODO: discuss threshold
df_stability['stability_cdp_system'] = df_stability \
    .apply(lambda x: x['eth_collateral_min'] >= 20e3, axis=1)

df_stability['kpi_stability'] = df_stability \
    .apply(lambda x: ( \
        x.stability_cdp_system == True and \
        x.stability_uniswap_liquidity == True and \
        x.stability_market_price == True and \
        x.stability_target_price == True) \
        , axis=1)

df_stability.query('kpi_stability == True').head(5)

,subset,run,market_price_min,market_price_max,target_price_min,target_price_max,RAI_balance_min,RAI_balance_max,eth_collateral_min,eth_collateral_max,stability_market_price,stability_target_price,stability_uniswap_liquidity,stability_cdp_system,kpi_stability
1,0,2,0.883572,5.177310,1.108362,4.553,7.418471e+06,1.449905e+07,53434.891273,154827.528922,True,True,True,True,True
2,0,3,1.061954,7.073488,1.318888,4.553,3.864257e+06,1.758366e+07,97111.387700,155413.059167,True,True,True,True,True
4,0,5,1.396159,9.729254,1.943650,4.553,9.195978e+06,2.501589e+07,29361.558765,154827.528922,True,True,True,True,True
6,1,2,0.588428,8.366921,0.466995,3.140,6.194085e+06,1.468085e+07,39222.130145,154827.528922,True,True,True,True,True
7,1,3,0.583197,5.712689,0.478570,3.140,3.390403e+06,2.184347e+07,63770.944535,154827.528922,True,True,True,True,True


time: 865 ms (started: 2021-02-12 10:51:03 +02:00)


## Volatility

**Volatility** threshold of market price: defined as the maximum value for the **standard deviation** computed. Defined relative to ETH price volatility. Definition: ratio of RAI price volatility / ETH price volatility is not to exceed 0.5.
  - over simulation period;
  - as moving average with 10-day window.

In [13]:
df_volatility_grouped = df_kpis.groupby(['subset', 'run'])

df_volatility_grouped = df_volatility_grouped.agg({'market_price': ['std'], 'eth_price': ['std']})
df_volatility_grouped.columns = ['market_price_std', 'eth_price_std']
df_volatility_grouped = df_volatility_grouped.reset_index()

df_volatility_grouped['volatility_ratio_simulation'] = \
    df_volatility_grouped[['market_price_std', 'eth_price_std']] \
    .apply(lambda x: x['market_price_std'] / x['eth_price_std'], axis=1)

df_volatility_grouped['kpi_volatility_simulation'] = df_volatility_grouped.apply(lambda x: x['volatility_ratio_simulation'] <= 0.5, axis=1)

df_volatility_grouped.head(5)

,subset,run,market_price_std,eth_price_std,volatility_ratio_simulation,kpi_volatility_simulation
0,0,1,1.525046,10.869019,0.140311,True
1,0,2,0.818362,14.633035,0.055926,True
2,0,3,0.603196,11.485685,0.052517,True
3,0,4,0.939423,12.187198,0.077083,True
4,0,5,0.324123,10.855959,0.029857,True


time: 386 ms (started: 2021-02-12 10:51:04 +02:00)


In [14]:
df_volatility_series = pd.DataFrame()
group = df_kpis.groupby(['subset', 'run'])

df_volatility_series['market_price_moving_average_std'] = group['market_price'].rolling(24*10, 1).std()
df_volatility_series['eth_price_moving_average_std'] = group['eth_price'].rolling(24*10, 1).std()
df_volatility_series.query('subset == 0 and run == 1').head(5)

market_price_moving_average_std  eth_price_moving_average_std
subset run                                                                 
0      1   0                              NaN                           NaN
           1                         0.000402                      2.022961
           2                         0.958166                      1.444726
           3                         0.830681                      1.979006
           4                         0.737611                      2.781775

time: 56.9 s (started: 2021-02-12 10:51:04 +02:00)


In [15]:
# danlessa was here 2.2s -> 1.2s
f = lambda x: x['market_price_moving_average_std'] / x['eth_price_moving_average_std']
df_volatility_series['volatility_ratio_window'] = df_volatility_series.parallel_apply(f, axis=1)
df_volatility_series.query('subset == 0 and run == 1').head(5)

market_price_moving_average_std  eth_price_moving_average_std  \
subset run                                                                    
0      1   0                              NaN                           NaN   
           1                         0.000402                      2.022961   
           2                         0.958166                      1.444726   
           3                         0.830681                      1.979006   
           4                         0.737611                      2.781775   

              volatility_ratio_window  
subset run                             
0      1   0                      NaN  
           1                 0.000199  
           2                 0.663216  
           3                 0.419747  
           4                 0.265158

time: 24.4 s (started: 2021-02-12 10:52:01 +02:00)


In [16]:
# danlessa was here. 2.8s -> 1.3s
f = lambda x: x['volatility_ratio_window'] != x['volatility_ratio_window'] or x['volatility_ratio_window'] <= 0.5
df_volatility_series['volatility_window_series'] = df_volatility_series.parallel_apply(f, axis=1)
df_volatility_series['volatility_window_mean'] = (df_volatility_series.groupby(['subset', 'run'])
                                                                           ['volatility_window_series']
                                                                          .transform(lambda x: x.mean()))
df_volatility_series.head(5)

market_price_moving_average_std  eth_price_moving_average_std  \
subset run                                                                    
0      1   0                              NaN                           NaN   
           1                         0.000402                      2.022961   
           2                         0.958166                      1.444726   
           3                         0.830681                      1.979006   
           4                         0.737611                      2.781775   

              volatility_ratio_window  volatility_window_series  \
subset run                                                        
0      1   0                      NaN                      True   
           1                 0.000199                      True   
           2                 0.663216                     False   
           3                 0.419747                      True   
           4                 0.265158                      True   

              volatility_window_mean  
subset run                            
0      1   0                0.988774  
           1                0.988774  
           2                0.988774  
           3                0.988774  
           4                0.988774

time: 26.6 s (started: 2021-02-12 10:52:25 +02:00)


In [17]:
df_volatility_series['volatility_window_mean'].describe()

count    8.711147e+06
mean     9.628357e-01
std      1.010184e-01
min      5.313585e-01
25%      9.997686e-01
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: volatility_window_mean, dtype: float64

time: 218 ms (started: 2021-02-12 10:52:52 +02:00)


In [18]:
df_volatility_series['kpi_volatility_window'] = df_volatility_series.groupby(['subset', 'run'])['volatility_window_mean'].transform(lambda x: x > 0.98)
df_volatility_series

market_price_moving_average_std  \
subset run                                            
0      1   0                                    NaN   
           1                               0.000402   
           2                               0.958166   
           3                               0.830681   
           4                               0.737611   
...                                             ...   
431    5   8711142                         0.132842   
           8711143                         0.132776   
           8711144                         0.132746   
           8711145                         0.132733   
           8711146                         0.132754   

                    eth_price_moving_average_std  volatility_ratio_window  \
subset run                                                                  
0      1   0                                 NaN                      NaN   
           1                            2.022961                 0.000199   
           2                            1.444726                 0.663216   
           3                            1.979006                 0.419747   
           4                            2.781775                 0.265158   
...                                          ...                      ...   
431    5   8711142                     12.061782                 0.011013   
           8711143                     12.157801                 0.010921   
           8711144                     12.260437                 0.010827   
           8711145                     12.390946                 0.010712   
           8711146                     12.510865                 0.010611   

                    volatility_window_series  volatility_window_mean  \
subset run                                                             
0      1   0                            True                0.988774   
           1                            True                0.988774   
           2                           False                0.988774   
           3                            True                0.988774   
           4                            True                0.988774   
...                                      ...                     ...   
431    5   8711142                      True                1.000000   
           8711143                      True                1.000000   
           8711144                      True                1.000000   
           8711145                      True                1.000000   
           8711146                      True                1.000000   

                    kpi_volatility_window  
subset run                                 
0      1   0                         True  
           1                         True  
           2                         True  
           3                         True  
           4                         True  
...                                   ...  
431    5   8711142                   True  
           8711143                   True  
           8711144                   True  
           8711145                   True  
           8711146                   True  

[8711147 rows x 6 columns]

time: 1.66 s (started: 2021-02-12 10:52:52 +02:00)


In [19]:
df_volatility_series.query('kpi_volatility_window == False')

market_price_moving_average_std  \
subset run                                            
2      1   4270                                 NaN   
           4271                            0.000402   
           4272                            0.958166   
           4273                            0.830681   
           4274                            0.737611   
...                                             ...   
427    4   6965215                         0.233979   
           6965216                         0.233376   
           6965217                         0.233575   
           6965218                         0.233641   
           6965219                         0.232993   

                    eth_price_moving_average_std  volatility_ratio_window  \
subset run                                                                  
2      1   4270                              NaN                      NaN   
           4271                         2.022961                 0.000199   
           4272                         1.444726                 0.663216   
           4273                         1.979006                 0.419747   
           4274                         2.781775                 0.265158   
...                                          ...                      ...   
427    4   6965215                      6.967666                 0.033581   
           6965216                      6.993090                 0.033372   
           6965217                      7.014977                 0.033297   
           6965218                      7.060526                 0.033091   
           6965219                      7.111248                 0.032764   

                    volatility_window_series  volatility_window_mean  \
subset run                                                             
2      1   4270                         True                0.963283   
           4271                         True                0.963283   
           4272                        False                0.963283   
           4273                         True                0.963283   
           4274                         True                0.963283   
...                                      ...                     ...   
427    4   6965215                      True                0.884517   
           6965216                      True                0.884517   
           6965217                      True                0.884517   
           6965218                      True                0.884517   
           6965219                      True                0.884517   

                    kpi_volatility_window  
subset run                                 
2      1   4270                     False  
           4271                     False  
           4272                     False  
           4273                     False  
           4274                     False  
...                                   ...  
427    4   6965215                  False  
           6965216                  False  
           6965217                  False  
           6965218                  False  
           6965219                  False  

[1424373 rows x 6 columns]

time: 927 ms (started: 2021-02-12 10:52:54 +02:00)


In [20]:
df_volatility_series['kpi_volatility_window'].value_counts()

True     7286774
False    1424373
Name: kpi_volatility_window, dtype: int64

time: 60.1 ms (started: 2021-02-12 10:52:55 +02:00)


## Merge KPI dataframes

In [21]:
# danlessa was here. 0.2s -> 80ms
cols_to_drop = {'volatility_ratio_window',
                'volatility_window_series',
                'market_price_moving_average_std',
                'eth_price_moving_average_std',
                'index'}

index_cols = ['run','subset']
dfs_to_join = [df_volatility_grouped, df_volatility_series, df_stability]

for i, df_to_join in enumerate(dfs_to_join):
    _df = df_to_join.reset_index()
    remaining_cols = list(set(_df.columns) - cols_to_drop)
    _df = (_df.reset_index()
              .loc[:, remaining_cols]
              .groupby(index_cols)
              .first()
          )
    dfs_to_join[i] = _df


df_kpis = (dfs_to_join[0].join(dfs_to_join[1], how='inner')
                         .join(dfs_to_join[2], how='inner')
          )

time: 840 ms (started: 2021-02-12 10:52:55 +02:00)


In [22]:
df_kpis['kpi_volatility'] = df_kpis.apply(lambda x: x['kpi_volatility_simulation'] and x['kpi_volatility_window'], axis=1)

time: 28.3 ms (started: 2021-02-12 10:52:56 +02:00)


In [23]:
df_kpis.query('kpi_volatility == False and kpi_stability == False')

kpi_volatility_simulation  market_price_std  \
run subset                                                
1   2                            True          2.174688   
    10                           True          2.497927   
    18                           True          2.255875   
    26                           True          2.210479   
    34                           True          2.586298   
...                               ...               ...   
4   411                          True          6.039560   
    417                         False          7.969418   
    419                          True          6.039560   
    425                         False          7.969418   
    427                          True          6.039560   

            volatility_ratio_simulation  eth_price_std  level_2  \
run subset                                                        
1   2                          0.193492      11.239179     4270   
    10                         0.222252      11.239179    30189   
    18                         0.207639      10.864421    56031   
    26                         0.196676      11.239179    81857   
    34                         0.230114      11.239179   107692   
...                                 ...            ...      ...   
4   411                        0.495566      12.187198  6891763   
    417                        0.653917      12.187198  6917689   
    419                        0.495566      12.187198  6926331   
    425                        0.653917      12.187198  6952257   
    427                        0.495566      12.187198  6960899   

            volatility_window_mean  kpi_volatility_window  market_price_max  \
run subset                                                                    
1   2                     0.963283                  False         69.558545   
    10                    0.932181                  False         53.635835   
    18                    0.967699                  False         52.682214   
    26                    0.963283                  False         70.923230   
    34                    0.932181                  False         56.098847   
...                            ...                    ...               ...   
4   411                   0.884517                  False         32.098646   
    417                   0.837306                  False         42.121102   
    419                   0.884517                  False         32.098646   
    425                   0.837306                  False         42.121102   
    427                   0.884517                  False         32.098646   

            stability_market_price  stability_cdp_system  market_price_min  \
run subset                                                                   
1   2                        False                  True          0.561593   
    10                       False                 False          0.444606   
    18                       False                  True          0.481341   
    26                       False                  True          0.478509   
    34                       False                 False          0.432107   
...                            ...                   ...               ...   
4   411                      False                  True          3.085153   
    417                      False                  True          3.140000   
    419                      False                  True          3.085153   
    425                      False                  True          3.140000   
    427                      False                  True          3.085153   

            RAI_balance_min  target_price_max  RAI_balance_max  \
run subset                                                       
1   2          1.844910e+04             4.553     1.450091e+07   
    10         2.212547e+04             4.553     1.450091e+07   
    18         2.437552e+04             4.553     1.450091e+07   

time: 35.3 ms (started: 2021-02-12 10:52:56 +02:00)


## Liquidity

**Liquidity** threshold of secondary market: defined as the maximum slippage value below which the controller is allowed to operate.
* __NB__: Threshold value will be determined by experimental outcomes, e.g. sample mean of the Monte Carlo outcomes of the slippage value when the system becomes unstable. Would like variance/std deviation of the Monte Carlo slippage series to be small (tight estimate), but can report both mean and variance as part of recommendations

In [24]:
critical_liquidity_threshold = None

time: 12.6 ms (started: 2021-02-12 10:52:56 +02:00)


In [25]:
df_liquidity = df[['subset', 'run', 'timestep', 'market_slippage']].copy()
df_liquidity = pd.merge(df_liquidity, df_kpis, how='inner', on=['subset', 'run'])
df_liquidity['market_slippage_abs'] = df_liquidity['market_slippage'].transform(lambda x: abs(x))
df_liquidity

,subset,run,timestep,market_slippage,kpi_volatility_simulation,market_price_std,volatility_ratio_simulation,eth_price_std,level_2,volatility_window_mean,kpi_volatility_window,market_price_max,stability_market_price,stability_cdp_system,market_price_min,RAI_balance_min,target_price_max,RAI_balance_max,eth_collateral_min,target_price_min,stability_target_price,eth_collateral_max,stability_uniswap_liquidity,kpi_stability,kpi_volatility,market_slippage_abs
0,0,1,0,0.000000,True,1.525046,0.140311,10.869019,0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,0.000000
1,0,1,1,0.000181,True,1.525046,0.140311,10.869019,0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,0.000181
2,0,1,2,NaN,True,1.525046,0.140311,10.869019,0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,NaN
3,0,1,3,-1.124826,True,1.525046,0.140311,10.869019,0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,1.124826
4,0,1,4,0.017344,True,1.525046,0.140311,10.869019,0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,0.017344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8711142,431,5,4316,NaN,True,0.537972,0.049555,10.855959,8706826,1.000000,True,5.787480,True,True,2.810935,7.837425e+06,3.140,1.900599e+07,43950.155901,3.140000,True,154827.528922,True,True,True,NaN
8711143,431,5,4317,0.003732,True,0.537972,0.049555,10.855959,8706826,1.000000,True,5.787480,True,True,2.810935,7.837425e+06,3.140,1.900599e+07,43950.155901,3.140000,True,154827.528922,True,True,True,0.003732
8711144,431,5,4318,0.003742,True,0.537972,0.049555,10.855959,8706826,1.000000,True,5.787480,True,True,2.810935,7.837425e+06,3.140,1.900599e+07,43950.155901,3.140000,True,154827.528922,True,True,True,0.003742
8711145,431,5,4319,NaN,True,0.537972,0.049555,10.855959,8706826,1.000000,True,5.787480,True,True,2.810935,7.837425e+06,3.140,1.900599e+07,43950.155901,3.140000,True,154827.528922,True,True,True,NaN


time: 3.48 s (started: 2021-02-12 10:52:56 +02:00)


In [26]:
df_liquidity.query('subset == 0')['market_slippage_abs'].describe([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.90])

count    9756.000000
mean        0.026507
std         0.187014
min         0.000000
10%         0.001578
20%         0.003212
30%         0.005043
40%         0.007176
50%         0.009700
60%         0.012536
70%         0.016728
80%         0.023129
90%         0.039628
max        14.658039
Name: market_slippage_abs, dtype: float64

time: 4.6 s (started: 2021-02-12 10:52:59 +02:00)


In [27]:
df_liquidity['market_slippage_percentile'] = df_liquidity.groupby(['subset', 'run'])['market_slippage'].transform(lambda x: x.quantile(.90))
df_liquidity

,subset,run,timestep,market_slippage,kpi_volatility_simulation,market_price_std,volatility_ratio_simulation,eth_price_std,level_2,volatility_window_mean,kpi_volatility_window,market_price_max,stability_market_price,stability_cdp_system,market_price_min,RAI_balance_min,target_price_max,RAI_balance_max,eth_collateral_min,target_price_min,stability_target_price,eth_collateral_max,stability_uniswap_liquidity,kpi_stability,kpi_volatility,market_slippage_abs,market_slippage_percentile
0,0,1,0,0.000000,True,1.525046,0.140311,10.869019,0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,0.000000,0.031924
1,0,1,1,0.000181,True,1.525046,0.140311,10.869019,0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,0.000181,0.031924
2,0,1,2,NaN,True,1.525046,0.140311,10.869019,0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,NaN,0.031924
3,0,1,3,-1.124826,True,1.525046,0.140311,10.869019,0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,1.124826,0.031924
4,0,1,4,0.017344,True,1.525046,0.140311,10.869019,0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,0.017344,0.031924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8711142,431,5,4316,NaN,True,0.537972,0.049555,10.855959,8706826,1.000000,True,5.787480,True,True,2.810935,7.837425e+06,3.140,1.900599e+07,43950.155901,3.140000,True,154827.528922,True,True,True,NaN,0.009279
8711143,431,5,4317,0.003732,True,0.537972,0.049555,10.855959,8706826,1.000000,True,5.787480,True,True,2.810935,7.837425e+06,3.140,1.900599e+07,43950.155901,3.140000,True,154827.528922,True,True,True,0.003732,0.009279
8711144,431,5,4318,0.003742,True,0.537972,0.049555,10.855959,8706826,1.000000,True,5.787480,True,True,2.810935,7.837425e+06,3.140,1.900599e+07,43950.155901,3.140000,True,154827.528922,True,True,True,0.003742,0.009279
8711145,431,5,4319,NaN,True,0.537972,0.049555,10.855959,8706826,1.000000,True,5.787480,True,True,2.810935,7.837425e+06,3.140,1.900599e+07,43950.155901,3.140000,True,154827.528922,True,True,True,NaN,0.009279


time: 3.19 s (started: 2021-02-12 10:53:04 +02:00)


In [28]:
%%capture
df_liquidity_failed = df_liquidity.query('kpi_volatility == False and kpi_stability == False')
df_liquidity_failed['market_slippage_percentile_mean'] = df_liquidity_failed.groupby(['subset'])['market_slippage_percentile'].transform(lambda x: x.mean())

time: 1.52 s (started: 2021-02-12 10:53:07 +02:00)


In [29]:
critical_liquidity_threshold = df_liquidity_failed['market_slippage_percentile_mean'].mean()
critical_liquidity_threshold

0.02364346333368304

time: 17 ms (started: 2021-02-12 10:53:09 +02:00)


In [30]:
df_liquidity_grouped = df_liquidity.groupby(['subset', 'run']).mean()
df_liquidity_grouped = df_liquidity_grouped.reset_index()
df_liquidity_grouped['kpi_liquidity'] = df_liquidity_grouped.apply(lambda x: x['market_slippage_percentile'] <= critical_liquidity_threshold, axis=1)
df_liquidity_grouped

,subset,run,timestep,market_slippage,kpi_volatility_simulation,market_price_std,volatility_ratio_simulation,eth_price_std,level_2,volatility_window_mean,kpi_volatility_window,market_price_max,stability_market_price,stability_cdp_system,market_price_min,RAI_balance_min,target_price_max,RAI_balance_max,eth_collateral_min,target_price_min,stability_target_price,eth_collateral_max,stability_uniswap_liquidity,kpi_stability,kpi_volatility,market_slippage_abs,market_slippage_percentile,kpi_liquidity
0,0,1,1113.0,-0.036437,True,1.525046,0.140311,10.869019,0.0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,0.067883,0.031924,False
1,0,2,2160.0,-0.001387,True,0.818362,0.055926,14.633035,1543746.0,1.000000,True,5.177310,True,True,0.883572,7.418471e+06,4.553,1.449905e+07,53434.891273,1.108362,True,154827.528922,True,True,True,0.016407,0.023197,True
2,0,3,2160.0,-0.002180,True,0.603196,0.052517,11.485685,3405462.0,1.000000,True,7.073488,True,True,1.061954,3.864257e+06,4.553,1.758366e+07,97111.387700,1.318888,True,155413.059167,True,True,True,0.019635,0.026033,False
3,0,4,2160.0,-0.003249,True,0.939423,0.077083,12.187198,5231877.0,1.000000,True,12.691588,True,True,0.537915,4.491708e+05,4.553,1.542804e+07,70859.763671,0.983880,True,154827.528922,False,False,True,0.039526,0.040180,False
4,0,5,2160.0,-0.000975,True,0.324123,0.029857,10.855959,6982504.0,1.000000,True,9.729254,True,True,1.396159,9.195978e+06,4.553,2.501589e+07,29361.558765,1.943650,True,154827.528922,True,True,True,0.009337,0.009389,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,431,1,2160.0,-0.000704,True,1.463367,0.110500,13.243125,1539425.0,1.000000,True,11.009319,True,True,3.108889,3.605694e+06,3.140,1.000091e+07,109627.886331,3.140000,True,154827.528922,True,True,True,0.013104,0.018857,True
2156,431,2,2160.0,-0.001260,True,2.813983,0.192303,14.633035,3401141.0,1.000000,True,17.029980,True,True,3.057557,4.468310e+06,3.140,1.251357e+07,130522.700116,3.140000,True,154827.528922,True,True,True,0.013178,0.019095,True
2157,431,3,2160.0,-0.001258,True,5.347825,0.465608,11.485685,5227556.0,1.000000,True,27.390752,True,True,2.978626,4.736914e+06,3.140,1.048611e+07,126102.117893,3.140000,True,154827.528922,True,True,True,0.012118,0.016607,True
2158,431,4,2160.0,-0.000430,True,1.247878,0.102393,12.187198,6978183.0,1.000000,True,9.461601,True,True,3.085153,5.606396e+06,3.140,1.140778e+07,129908.524813,3.140000,True,154827.528922,True,True,True,0.010711,0.016329,True


time: 8.62 s (started: 2021-02-12 10:53:09 +02:00)


In [31]:
df_kpis = df_liquidity_grouped[['subset', 'run', 'kpi_stability', 'kpi_volatility', 'kpi_liquidity']]
df_kpis = df_kpis.groupby(['subset', 'run']).first()

time: 11.1 ms (started: 2021-02-12 10:53:17 +02:00)


In [32]:
print(f'''
{round(df_kpis.query('kpi_stability == True and kpi_volatility == True and kpi_liquidity == True').count().iloc[0]*100/df_kpis.count().iloc[0])}% successful KPIs
''')


53% successful KPIs

time: 13.3 ms (started: 2021-02-12 10:53:17 +02:00)


## Save KPI Results

In [33]:
df_kpis.to_pickle('experiments/system_model_v3/experiment_monte_carlo/kpi_dataset.pickle')

time: 13.1 ms (started: 2021-02-12 10:53:17 +02:00)


In [34]:
df_kpis = pd.read_pickle('experiments/system_model_v3/experiment_monte_carlo/kpi_dataset.pickle')

time: 12.5 ms (started: 2021-02-12 10:53:17 +02:00)


# Sensitivity Analysis

In [35]:
df.head(5)

,timedelta,cumulative_time,timestamp,eth_price,liquidity_demand,liquidity_demand_mean,eth_collateral,eth_locked,eth_freed,principal_debt,rai_drawn,rai_wiped,system_revenue,stability_fee,market_price,market_price_twap,target_price,target_rate,eth_return,eth_gross_return,expected_market_price,expected_debt_price,error_star,error_star_integral,market_slippage,RAI_balance,ETH_balance,UNI_supply,simulation,subset,run,substep,timestep,eth_collateral_value,collateralization_ratio,ki,kp,liquidation_ratio,controller_enabled,control_period,liquidity_demand_shock,arbitrageur_considers_liquidation_ratio,rescale_target_price,target_price_scaled
0,0,0,2017-01-01 00:00:00,294.069151,1.000000,1.000000,154827.528922,154827.528922,0.000000,1.000000e+07,1.000000e+07,0.000000e+00,0.0,1.585490e-10,3.140000,0.0,3.140000,0.0,0.000000,0.0,3.140000,3.14,0.00,0.0,0.000000,1.000000e+07,106777.606153,1.000000e+07,0,0,1,0,0,4.553000e+07,1.450000,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,4.553
1,3600,3600,2017-01-01 01:00:00,291.208252,906.674484,453.837242,154827.528922,154827.528922,0.000000,1.450000e+07,1.450000e+07,0.000000e+00,0.0,1.585490e-10,3.139432,0.0,2.165517,0.0,-0.009729,1.0,3.231162,3.14,3.14,5652.0,0.000181,1.450091e+07,73703.760510,1.000000e+07,0,0,1,18,1,4.508705e+07,1.435893,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,3.140
2,3600,7200,2017-01-01 02:00:00,292.287792,-326.024615,63.906314,107503.536596,154827.528922,47323.992326,9.970037e+06,1.450000e+07,4.529963e+06,0.0,1.585490e-10,1.480124,0.0,2.165517,0.0,0.003707,1.0,1.523521,3.14,3.14,11458.0,NaN,9.970618e+06,107288.025922,9.999775e+06,0,0,1,18,2,3.142197e+07,1.455375,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,3.140
3,3600,10800,2017-01-01 03:00:00,289.343698,209.658450,136.782382,107503.536596,154827.528922,47323.992326,1.000700e+07,1.453696e+07,4.529963e+06,0.0,1.585490e-10,3.145007,0.0,2.165517,0.0,-0.010073,1.0,3.236929,3.14,3.14,11616.0,-1.124826,1.000779e+07,106890.741521,9.999775e+06,0,0,1,18,3,3.110547e+07,1.435395,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,3.140
4,3600,14400,2017-01-01 04:00:00,286.827771,-79.844839,28.468771,107503.536596,154827.528922,47323.992326,9.906201e+06,1.453696e+07,4.630759e+06,0.0,1.585490e-10,3.090459,0.0,2.165517,0.0,-0.008695,1.0,3.181390,3.14,3.14,11620.0,0.017344,9.906911e+06,107982.421498,9.999775e+06,0,0,1,18,4,3.083500e+07,1.437392,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,3.140


time: 30.8 ms (started: 2021-02-12 10:53:17 +02:00)


In [36]:
df_kpis.head(5)

kpi_stability  kpi_volatility  kpi_liquidity
subset run                                              
0      1            False            True          False
       2             True            True           True
       3             True            True          False
       4            False            True          False
       5             True            True           True

time: 12.2 ms (started: 2021-02-12 10:53:17 +02:00)


In [37]:
df_sensitivity = pd.merge(df, df_kpis, on=['run','subset'], how='inner')
df_sensitivity

,timedelta,cumulative_time,timestamp,eth_price,liquidity_demand,liquidity_demand_mean,eth_collateral,eth_locked,eth_freed,principal_debt,rai_drawn,rai_wiped,system_revenue,stability_fee,market_price,market_price_twap,target_price,target_rate,eth_return,eth_gross_return,expected_market_price,expected_debt_price,error_star,error_star_integral,market_slippage_x,RAI_balance,ETH_balance,UNI_supply,simulation,subset,run,substep,timestep_x,eth_collateral_value,collateralization_ratio,ki,kp,liquidation_ratio,controller_enabled,control_period,liquidity_demand_shock,arbitrageur_considers_liquidation_ratio,rescale_target_price,target_price_scaled,timestep_y,market_slippage_y,kpi_volatility_simulation,market_price_std,volatility_ratio_simulation,eth_price_std,level_2,volatility_window_mean,kpi_volatility_window,market_price_max,stability_market_price,stability_cdp_system,market_price_min,RAI_balance_min,target_price_max,RAI_balance_max,eth_collateral_min,target_price_min,stability_target_price,eth_collateral_max,stability_uniswap_liquidity,kpi_stability,kpi_volatility,market_slippage_abs,market_slippage_percentile,kpi_liquidity
0,0,0,2017-01-01 00:00:00,294.069151,1.000000,1.000000,154827.528922,154827.528922,0.000000,1.000000e+07,1.000000e+07,0.000000e+00,0.0,1.585490e-10,3.140000,0.000000,3.140000,0.0,0.000000,0.0,3.140000,3.14,0.000000,0.000000,0.000000,1.000000e+07,106777.606153,1.000000e+07,0,0,1,0,0,4.553000e+07,1.450000,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,4.553,1113.0,-0.036437,True,1.525046,0.140311,10.869019,0.0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,0.067883,0.031924,False
1,3600,3600,2017-01-01 01:00:00,291.208252,906.674484,453.837242,154827.528922,154827.528922,0.000000,1.450000e+07,1.450000e+07,0.000000e+00,0.0,1.585490e-10,3.139432,0.000000,2.165517,0.0,-0.009729,1.0,3.231162,3.14,3.140000,5652.000000,0.000181,1.450091e+07,73703.760510,1.000000e+07,0,0,1,18,1,4.508705e+07,1.435893,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,3.140,1113.0,-0.036437,True,1.525046,0.140311,10.869019,0.0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,0.067883,0.031924,False
2,3600,7200,2017-01-01 02:00:00,292.287792,-326.024615,63.906314,107503.536596,154827.528922,47323.992326,9.970037e+06,1.450000e+07,4.529963e+06,0.0,1.585490e-10,1.480124,0.000000,2.165517,0.0,0.003707,1.0,1.523521,3.14,3.140000,11458.000000,NaN,9.970618e+06,107288.025922,9.999775e+06,0,0,1,18,2,3.142197e+07,1.455375,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,3.140,1113.0,-0.036437,True,1.525046,0.140311,10.869019,0.0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,0.067883,0.031924,False
3,3600,10800,2017-01-01 03:00:00,289.343698,209.658450,136.782382,107503.536596,154827.528922,47323.992326,1.000700e+07,1.453696e+07,4.529963e+06,0.0,1.585490e-10,3.145007,0.000000,2.165517,0.0,-0.010073,1.0,3.236929,3.14,3.140000,11616.000000,-1.124826,1.000779e+07,106890.741521,9.999775e+06,0,0,1,18,3,3.110547e+07,1.435395,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,3.140,1113.0,-0.036437,True,1.525046,0.140311,10.869019,0.0,0.988774,True,34.406646,False,True,0.253244,1.916061e+04,4.553,1.450091e+07,24243.557193,1.538902,True,154827.528922,False,False,True,0.067883,0.031924,False
4,3600,14400,2017-01-01 04:00:00,286.827771,-79.844839,28.468771,107503.536596,154827.528922,47323.992326,9.906201e+06,1.453696e+07,4.630759e+06,0.0,1.585490e-10,3.090459,0.000000,2.165517,0.0,-0.008695,1.0,3.181390,3.14,3.140000,11620.000000,0.017344,9.906911e+06,107982.421498,9.999775e+06,0,0,1,18,4,3.083500e+07,1.437392,-5.000000e-09,2.000000e-07,1.45,True,3600.0,True,True,True,3.140,1113.0,-0.036437,True,1.525046,0.140311,10.869019,0.0,0.988774,True,34.406646,False,True,0.253244,

time: 1min 7s (started: 2021-02-12 10:53:17 +02:00)


In [38]:
df_sensitivity = df_sensitivity.reset_index()

time: 14.7 s (started: 2021-02-12 10:54:25 +02:00)


In [39]:
# Controller on / off
# Rescale and arb. considers lr; both true or false
# liquidity_demand_shock 10 /50 %

time: 39 ms (started: 2021-02-12 10:54:40 +02:00)


In [40]:
control_params = [
    'ki',
    'kp',
    'control_period',
]

time: 12.6 ms (started: 2021-02-12 10:54:40 +02:00)


In [ ]:
from cadcad_machine_search.visualizations import kpi_sensitivity_plot

# df = dataframe with KPI values stored as columns, with runs as rows
# control_params = column names in df containing control parameter values for each run

# kpis = {
#     'volatility_simulation'        : lambda df: df['volatility_ratio_simulation'],
#     'volatility_window_mean'       : lambda df: df['volatility_window_mean'],
#     'market_price_max'             : lambda df: df['market_price'].max(),
#     'market_price_min'             : lambda df: df['market_price'].min(),
#     'redemption_price_max'         : lambda df: df['target_price_scaled'].max(),
#     'redemption_price_min'         : lambda df: df['target_price_scaled'].min(),
#     'rai_balance_uniswap_min'      : lambda df: df['RAI_balance'].min(),
#     'cdp_collateral_balance_min'   : lambda df: df['eth_collateral'].min(),
#     'price_change_percentile_mean' : lambda df: critical_liquidity_threshold
# }

goals = {
    'low_volatility'  : lambda metrics: metrics['kpi_volatility'],
    'high_stability'  : lambda metrics: metrics['kpi_stability'],
    'liquidity_threshold': lambda metrics: metrics['kpi_liquidity'],
}

#     'controller_enabled',
#     'liquidity_demand_shock',
#     'arbitrageur_considers_liquidation_ratio',
#     'rescale_target_price'

kpi_sensitivity_plot(df_sensitivity, goals['low_volatility'], control_params)

# for scenario in df_sensitivity['controller_enabled'].unique():
#     df = df_sensitivity.query(f'controller_enabled == {scenario}')
#     for goal in goals:
#         kpi_sensitivity_plot(df, goals[goal], control_params)

# for scenario in df_sensitivity['liquidity_demand_shock'].unique():
#     df = df_sensitivity.query(f'liquidity_demand_shock == {scenario}')
#     for goal in goals:
#         kpi_sensitivity_plot(df, goals[goal], control_params)

# TODO:
# for scenario in df_sensitivity['controller_enabled'].unique():
#     df = df_sensitivity.query(f'controller_enabled == {scenario}')
#     for goal in goals:
#         kpi_sensitivity_plot(df, goals[goal], control_params)

In [ ]:
from cadcad_machine_search.visualizations import plot_goal_ternary

# df = dataframe with KPI values stored as columns, with runs as rows
# control_params = column names in df containing control parameter values for each run

kpis = {
    'volatility_simulation'        : lambda df: df['volatility_ratio_simulation'],
    'volatility_window_mean'            : lambda df: df['volatility_ratio_window'].mean(),
    'market_price_max'             : lambda df: df['market_price'].max(),
    'market_price_min'             : lambda df: df['market_price'].min(),
    'redemption_price_max'         : lambda df: df['target_price_scaled'].max(),
    'redemption_price_min'         : lambda df: df['target_price_scaled'].min(),
    'rai_balance_uniswap_min'      : lambda df: df['RAI_balance'].min(),
    'cdp_collateral_balance_min'   : lambda df: df['eth_collateral'].min(),
    'price_change_percentile_mean' : lambda df: critical_liquidity_threshold
}

goals = {
    'low_volatility' : lambda metrics: -0.5 * ( metrics['volatility_simulation'] +
                    metrics['price_change_percentile_mean'] ),
    'high_stability'  : lambda metrics: -(1/6) * ( metrics['market_price_max'] + 
                    1 / metrics['market_price_min'] + metrics['redemption_price_max'] +
                    1 / metrics['redemption_price_min'] + 1 / metrics['rai_balance_uniswap_min'] +
                    1 / metrics['cdp_collateral_balance_min'] ),
    'liquidity'  : lambda metrics: -metrics['price_change_percentile_mean'],
    'combined'   : lambda goals: goals[0] + goals[1] + goals[2]
}


for scenario in df_sensitivity['controller_enabled'].unique():
    df = df_sensitivity.query(f'controller_enabled == {scenario}')
    plot_goal_ternary(df, kpis, goals, control_params)

for scenario in df_sensitivity['liquidity_demand_shock'].unique():
    df = df_sensitivity.query(f'liquidity_demand_shock == {scenario}')
    plot_goal_ternary(df, kpis, goals, control_params)   

# TODO:
# for scenario in df_sensitivity['controller_enabled'].unique():
#     df = df_sensitivity.query(f'controller_enabled == {scenario}')
#     for goal in goals:
#         kpi_sensitivity_plot(df, goals[goal], control_params)

# TODO: save both for presentation

In [ ]:
# TODO:
# Kp, Ki, control period
# Alpha

# All pass:
# Minimum volatility (values)
# Minimum liquidity (values)

In [ ]:
# TODO: Timeseries 

In [ ]:
# TODO: Experiment execution metrics

# Runtime (final + sanity checking)
# Number of exp.
# Number of parameters per exp.
# MC runs
# Timesteps * runs * subsets